In [ ]:
# <<<<<<< Setting >>>>>>>

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import scanpy as sc
import gseapy
import scvi
import scipy.sparse as sp
import math
import anndata as ad

import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['font.family'] = 'Arial'
plt.rcParams["pdf.fonttype"] = 42          
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import matplotlib.patheffects as pe

sc.settings.verbosity = 3 

import os
print(os.getcwd())
cwd = f"{os.getcwd()}"

sc.settings.set_figure_params(dpi=200)

print(matplotlib.rcParams["font.family"])

In [ ]:
# <<<<<<< selected gene count >>>>>>>

h5ad_path = f"{cwd}/Neuron2022_Human_TG.h5ad"
out_xlsx  = f"{cwd}/selected_gene_counts.xlsx"

genes_want = ["TRAC", "TRBC1", "TRBC2", "CD4", "CD8A", "CD8B"]  

use_raw = True          
layer_name = None       

adata = sc.read_h5ad(h5ad_path)

if use_raw and adata.raw is not None:
    M = adata.raw.X
    var_index = adata.raw.var_names
else:
    if layer_name is not None:
        if layer_name not in adata.layers:
            raise ValueError(f"layer '{layer_name}' did not found. Available: {list(adata.layers.keys())}")
        M = adata.layers[layer_name]
    else:
        M = adata.X
    var_index = adata.var_names

var_index = pd.Index(var_index)
found_mask = var_index.isin(genes_want)
genes_found = var_index[found_mask].tolist()
genes_missing = sorted(set(genes_want) - set(genes_found))

if len(genes_found) == 0:
    raise ValueError("The specified gene was not found. Please check the spelling in var_names.")

gene_idx = var_index.get_indexer(genes_found)

if sp.issparse(M):
    sub = M[:, gene_idx].toarray()
else:
    sub = M[:, gene_idx]

df = pd.DataFrame(sub, index=adata.obs_names, columns=genes_found)

with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="counts_cells_x_genes", index=True)

    pd.DataFrame({"missing_genes": genes_missing}).to_excel(
        writer, sheet_name="missing_genes", index=False
    )

print("done:", out_xlsx)


In [ ]:
gene_names = adata.var_names.to_list()

In [ ]:
# <<<<<<< Cell count >>>>>>>

adata = sc.read_h5ad(f"{cwd}/Neuron2022_Human_TG.h5ad")

def get_expr_or_zero(adata, gene):
    if gene in adata.var_names:
        x = adata[:, gene].X
        expr = x.toarray().flatten() if not isinstance(x, np.ndarray) else x.flatten()
        return expr > 0
    else:
        print(f"[skip] {gene} is not exist")
        return np.zeros(adata.n_obs, dtype=bool)
    
def get_multi_gene_expr(adata, genes):
    return np.logical_or.reduce([get_expr_or_zero(adata, g) for g in genes])

exp_TRA = get_multi_gene_expr(adata, ["TRAC"])
exp_TRB = get_multi_gene_expr(adata, ["TRBC1", "TRBC2"])
exp_CD4 = get_multi_gene_expr(adata, ["CD4"])
exp_CD8 = get_multi_gene_expr(adata, ["CD8A", "CD8B", "CD8B2"])
exp_CD3 = get_multi_gene_expr(adata, ["CD3D", "CD3E", "CD3G"])

exp_CSF1R = get_multi_gene_expr(adata, ["CSF1R"])
exp_ZBTB46 = get_multi_gene_expr(adata, ["ZBTB46"])
exp_HLA_DRA = get_multi_gene_expr(adata, ["HLA-DRA"])
exp_H2AB1 = get_multi_gene_expr(adata, ["H2AB1"])
exp_LYZ = get_multi_gene_expr(adata, ["LYZ"])
exp_MS4A6A = get_multi_gene_expr(adata, ["MS4A6A"]) 
exp_AIF1 = get_multi_gene_expr(adata, ["AIF1"])
exp_MERTK = get_multi_gene_expr(adata, ["MERTK"])
exp_TMEM119 = get_multi_gene_expr(adata, ["TMEM119"])
exp_SOX9 = get_multi_gene_expr(adata, ["SOX9"])

exp_CEACAM8 = get_multi_gene_expr(adata, [
    "CEACAM8", "LY6G6C", "LY6G6D", "LY6G6E", "LY6G6F", 
])

exp_TRGV = get_multi_gene_expr(adata, [
    "TRGV1", "TRGV2", "TRGV3", "TRGV4", "TRGV5", "TRGV8", "TRGV9", "TRGV10", "TRGV11"
])

exp_TRDV = get_multi_gene_expr(adata, [
    "TRDV1", "TRDV2", "TRDV3"
])

exp_IFNG = get_multi_gene_expr(adata, ["IFNG"])
exp_PTPRC = get_multi_gene_expr(adata, ["PTPRC"])



adata.obs["CD4+"] = exp_CD4 
adata.obs["CD8+"] = exp_CD8 
adata.obs["CD4+_CD8-"] = exp_CD4 & ~exp_CD8
adata.obs["CD4-_CD8+"] = exp_CD8 & ~exp_CD4
adata.obs["CD4+_CD8+"] = exp_CD4 & exp_CD8
adata.obs["CD4-_CD8-"] = ~exp_CD4 & ~exp_CD8

adata.obs["TRA+"] = exp_TRA
adata.obs["TRB+"] = exp_TRB
adata.obs["TRA+_TRB+"] = exp_TRA & exp_TRB
adata.obs["(TRA+_or_TRB+)"] = exp_TRA | exp_TRB

adata.obs["TRA+_CD4+"] = exp_TRA & exp_CD4
adata.obs["TRA+_CD8+"] = exp_TRA & exp_CD8
adata.obs["TRB+_CD4+"] = exp_TRB & exp_CD4
adata.obs["TRB+_CD8+"] = exp_TRB & exp_CD8

adata.obs["TRA+_TRB+_CD4+_CD8-"] = exp_TRA & exp_TRB & exp_CD4 & ~exp_CD8
adata.obs["TRA+_TRB+_CD4-_CD8+"] = exp_TRA & exp_TRB & exp_CD8 & ~exp_CD4
adata.obs["TRA+_TRB+_CD4+_CD8+"] = exp_TRA & exp_TRB & exp_CD4 & exp_CD8
adata.obs["TRA+_TRB+_CD4-_CD8-"] = exp_TRA & exp_TRB & ~exp_CD4 & ~exp_CD8

adata.obs["(TRA+_or_TRB+)_CD4+_CD8-"] = (exp_TRA | exp_TRB) & exp_CD4 & ~exp_CD8
adata.obs["(TRA+_or_TRB+)_CD4-_CD8+"] = (exp_TRA | exp_TRB) & exp_CD8 & ~exp_CD4
adata.obs["(TRA+_or_TRB+)_CD4+_CD8+"] = (exp_TRA | exp_TRB) & exp_CD4 & exp_CD8
adata.obs["(TRA+_or_TRB+)_CD4-_CD8-"] = (exp_TRA | exp_TRB) & ~exp_CD4 & ~exp_CD8

adata.obs["CD45+_(TRA+_or_TRB+)_CD4+_CD8-"] = exp_PTPRC & (exp_TRA | exp_TRB) & exp_CD4 & ~exp_CD8
adata.obs["CD45+_(TRA+_or_TRB+)_CD4-_CD8+"] = exp_PTPRC & (exp_TRA | exp_TRB) & exp_CD8 & ~exp_CD4

adata.obs["CD3+"] = exp_CD3
adata.obs["CD3+_CD4+"] = exp_CD3 & exp_CD4 
adata.obs["CD3+_CD8+"] = exp_CD3 & exp_CD8 
adata.obs["CD3+_TRA+"] = exp_CD3 & exp_TRA
adata.obs["CD3+_TRB+"] = exp_CD3 & exp_TRB

adata.obs["CD45+"] = exp_PTPRC
adata.obs["CD45+_CD4+"] = exp_PTPRC & exp_CD4 
adata.obs["CD45+_CD8+"] = exp_PTPRC & exp_CD8 
adata.obs["CD45+_TRA+_TRB-"] = exp_PTPRC & exp_TRA & ~exp_TRB
adata.obs["CD45+_TRA-_TRB+"] = exp_PTPRC & ~exp_TRA & exp_TRB
adata.obs["CD45+_TRA+_TRB+"] = exp_PTPRC & exp_TRA & exp_TRB
adata.obs["CD45+_(TRA+_or_TRB+)"] = exp_PTPRC & (exp_TRA | exp_TRB)

adata.obs["IFNG_positive"] = exp_IFNG

adata.obs["monocyte_dendritic_cells_macrophages"] = exp_CSF1R | exp_ZBTB46 | exp_HLA_DRA | exp_LYZ | exp_MS4A6A | exp_H2AB1 | exp_MERTK
adata.obs["microgria"] = exp_AIF1 | exp_TMEM119
adata.obs["astrocyte"] = exp_SOX9
adata.obs["neutrophil"] = exp_CEACAM8
adata.obs["gd_T_cell"] = exp_TRGV | exp_TRDV
adata.obs["gate_out_cell"] = exp_CSF1R | exp_ZBTB46 | exp_HLA_DRA | exp_LYZ | exp_MS4A6A | exp_H2AB1 | exp_MERTK | exp_AIF1 | exp_TMEM119 | exp_SOX9 | exp_CEACAM8 | exp_TRGV | exp_TRDV

adata.obs["CD4+_CD8-__gateout"] = exp_CD4 & ~exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["CD4-_CD8+__gateout"] = exp_CD8 & ~exp_CD4 & ~adata.obs["gate_out_cell"]
adata.obs["CD4+_CD8+__gateout"] = exp_CD4 & exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["CD4-_CD8-__gateout"] = ~exp_CD4 & ~exp_CD8 & ~adata.obs["gate_out_cell"]

adata.obs["TRA+__gateout"] = exp_TRA & ~adata.obs["gate_out_cell"]
adata.obs["TRB+__gateout"] = exp_TRB & ~adata.obs["gate_out_cell"]
adata.obs["TRA+_TRB+__gateout"] = exp_TRA & exp_TRB & ~adata.obs["gate_out_cell"]
adata.obs["(TRA+_or_TRB+)__gateout"] = exp_TRA | exp_TRB & ~adata.obs["gate_out_cell"]

adata.obs["TRA+_CD4+__gateout"] = exp_TRA & exp_CD4 & ~adata.obs["gate_out_cell"]
adata.obs["TRA+_CD8+__gateout"] = exp_TRA & exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["TRB+_CD4+__gateout"] = exp_TRB & exp_CD4 & ~adata.obs["gate_out_cell"]
adata.obs["TRB+_CD8+__gateout"] = exp_TRB & exp_CD8 & ~adata.obs["gate_out_cell"]

adata.obs["TRA+_TRB+_CD4+_CD8-__gateout"] = exp_TRA & exp_TRB & exp_CD4 & ~exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["TRA+_TRB+_CD4-_CD8+__gateout"] = exp_TRA & exp_TRB & exp_CD8 & ~exp_CD4 & ~adata.obs["gate_out_cell"]
adata.obs["TRA+_TRB+_CD4+_CD8+__gateout"] = exp_TRA & exp_TRB & exp_CD4 & exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["TRA+_TRB+_CD4-_CD8-__gateout"] = exp_TRA & exp_TRB & ~exp_CD4 & ~exp_CD8 & ~adata.obs["gate_out_cell"]

adata.obs["(TRA+_or_TRB+)_CD4+_CD8-__gateout"] = (exp_TRA | exp_TRB) & exp_CD4 & ~exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["(TRA+_or_TRB+)_CD4-_CD8+__gateout"] = (exp_TRA | exp_TRB) & exp_CD8 & ~exp_CD4 & ~adata.obs["gate_out_cell"]
adata.obs["(TRA+_or_TRB+)_CD4+_CD8+__gateout"] = (exp_TRA | exp_TRB) & exp_CD4 & exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["(TRA+_or_TRB+)_CD4-_CD8-__gateout"] = (exp_TRA | exp_TRB) & ~exp_CD4 & ~exp_CD8 & ~adata.obs["gate_out_cell"]

adata.obs["CD3+__gateout"] = exp_CD3 & ~adata.obs["gate_out_cell"]
adata.obs["CD3+_CD4+__gateout"] = exp_CD3 & exp_CD4 & ~adata.obs["gate_out_cell"]
adata.obs["CD3+_CD8+__gateout"] = exp_CD3 & exp_CD8 & ~adata.obs["gate_out_cell"]
adata.obs["CD3+_TRA+__gateout"] = exp_CD3 & exp_TRA & ~adata.obs["gate_out_cell"]
adata.obs["CD3+_TRB+__gateout"] = exp_CD3 & exp_TRB & ~adata.obs["gate_out_cell"]

adata.obs["CD45+__gateout"] = exp_PTPRC & ~adata.obs["gate_out_cell"]
adata.obs["CD45+_CD4+__gateout"] = exp_PTPRC & exp_CD4  & ~adata.obs["gate_out_cell"]
adata.obs["CD45+_CD8+__gateout"] = exp_PTPRC & exp_CD8  & ~adata.obs["gate_out_cell"]
adata.obs["CD45+_TRA+_TRB-__gateout"] = exp_PTPRC & exp_TRA & ~exp_TRB & ~adata.obs["gate_out_cell"]
adata.obs["CD45+_TRA-_TRB+__gateout"] = exp_PTPRC & ~exp_TRA & exp_TRB & ~adata.obs["gate_out_cell"]
adata.obs["CD45+_TRA+_TRB+__gateout"] = exp_PTPRC & exp_TRA & exp_TRB & ~adata.obs["gate_out_cell"]
adata.obs["CD45+_(TRA+_or_TRB+)__gateout"] = exp_PTPRC & (exp_TRA | exp_TRB) & ~adata.obs["gate_out_cell"]


total_cells = adata.n_obs
print("total_cells:", total_cells)

for col in adata.obs.columns:
    if adata.obs[col].dtype == bool:   
        print(f"{col}: {adata.obs[col].sum()}")



In [ ]:
adata

In [ ]:
# <<<<<<< Define subsets for analysis >>>>>>>

subset1 = adata.copy()  

#subset2 = adata[adata.obs["CD4+_CD8-__gateout"]].copy()

#subset3 = adata[adata.obs["CD4-_CD8+__gateout"]].copy()

#subset4 = adata[adata.obs["CD4+_CD8-"]].copy()

#subset5 = adata[adata.obs["CD4-_CD8+"]].copy()

#subset6 = adata[(adata.obs["CD4+_CD8-"]) |(adata.obs["CD4-_CD8+"]) | (adata.obs["CD4+_CD8+"])].copy()

#subset7 = adata[adata.obs["TRA+"]].copy()

#subset8 = adata[adata.obs["TRB+"]].copy()

#subset9 = adata[adata.obs["(TRA+_or_TRB+)"]].copy()

subset10 = adata[adata.obs["CD45+"]].copy()



subsets = [
    ("human_all_cells", subset1),
#    ("human_CD4+_CD8-_gateout", subset2),
#    ("human_CD4-_CD8+_gateout", subset3),
#    ("human_CD4+_CD8-", subset4),
#    ("human_CD4-_CD8+", subset5),
#    ("human_CD4+_or_CD8+", subset6),
#    ("human_TRA+", subset7),
#    ("human_TRB+", subset8),
#    ("human_TRA+_or_TRB+", subset9),
    ("human_CD45+", subset10)
]


In [ ]:

human_gene_list = [
    "CD3D", "CD3E", "CD3G", "CD4", "CD8A", "CD8B", "CD44", "SELL", "CCR7", "KLF2", 
    "S1PR1", "NR4A1", "NR4A2", "NR4A3", "JAK1", "JAK2", "JAK3","ISG15", "ISG20", "RORA", 
    "ZNF683", "TBX21", "TCF7", "TOX", "GATA3", "CXCR3","CXCR6", "CD69", "IFNG", "IL10", 
    "IL21", "CXCR5", "FOXP3", "CTLA4", "CX3CR1", "GZMA", "GZMB", "GZMK", "PRF1", "CCL5", 
    "ITGAE", "ITGA1"
]

existing_genes = [g for g in human_gene_list if g in adata.var_names]

missing_genes = [g for g in human_gene_list if g not in adata.var_names]

if missing_genes:
    print("The following genes were not found:")
    print(missing_genes)



CD4_list = [
    "ISG15", "ISG20", "IFI6", "IFIT5", "IFI35", "IFITM1", "IRF7", "GBP4", "CXCL16"
]

CD4_genes = [g for g in CD4_list if g in adata.var_names]

CD4_missing_genes = [g for g in CD4_list if g not in adata.var_names]

if CD4_missing_genes:
    print("The following genes were not found:")
    print(CD4_missing_genes)



CD8_list = [
    "ISG15", "ISG20", "CCL5", "IFI44", "OAS1"
]

CD8_genes = [g for g in CD8_list if g in adata.var_names]

CD8_missing_genes = [g for g in CD8_list if g not in adata.var_names]

if CD8_missing_genes:
    print("The following genes were not found:")
    print(CD8_missing_genes)



In [ ]:
# <<<<<<< subset analysis >>>>>>>

min_cells = 3  
for name, subset in subsets:

    print(f"Processing: {name}(Num of cells: {subset.n_obs})")

    if subset.n_obs < min_cells:
        print(f"Skip: {name} has {subset.n_obs} cells, which is below {min_cells}.")
        continue

    try:

        # ======= pre-analysis of subset =======

        sc.pp.normalize_total(subset, target_sum=1e4)

        subset.raw = subset

        sc.pp.log1p(subset)

        sc.pp.highly_variable_genes(subset, n_top_genes=2000, subset=False)

        if "highly_variable" not in subset.var or subset.var["highly_variable"].sum() == 0:
            print(f"Skip: No high-variability genes were detected in {name}")
            continue

        subset = subset[:, subset.var["highly_variable"]].copy()

        sc.pp.scale(subset, max_value=10)

        sc.tl.pca(subset)

        sc.pp.neighbors(subset)

        sc.tl.umap(subset)

        resolution = 1.1

        sc.tl.leiden(subset, resolution)

        print(subset.X[:5, :5])  
        print(subset.var.head())  
        print("High variable genes:", subset.var["highly_variable"].sum())
        print(subset.obsm.keys())  
        print(subset.obsm['X_pca'][:5, :5])  
        print(subset.obsm['X_pca'].shape)  
        print(subset.uns['neighbors'].keys())
        print(subset.obsp['distances'].shape)
        print(subset.obsp['connectivities'].shape)
        print(subset.obsm.keys())  
        print(subset.obsm['X_umap'][:5, :])  
        print(subset.obs.keys())  
        print(subset.obs['leiden'].value_counts()) 

        # ======= Cluster =======
        print(f"Processing: {name} - Cluster analysis")

        #UMAP
        sc.pl.umap(
            subset, 
            color="leiden", 
            show=True, 
            save = f"_{name}_subset_cluster_{resolution}_UMAP.pdf"
        )

        #DEG
        sc.tl.rank_genes_groups(
            subset, 
            groupby="leiden", 
            method="wilcoxon"
        )
        sc.pl.rank_genes_groups(
            subset, 
            n_genes=25, 
            sharey=False, 
            show=False, 
            save = f"_{name}_subset_cluster_{resolution}_DEG_fig.pdf"
        )

        deg_result = subset.uns["rank_genes_groups"]
        groups = deg_result["names"].dtype.names

        with pd.ExcelWriter(f"{cwd}/figures/{name}_subset_cluster_{resolution}_DEG_results.xlsx") as writer:
            for group in groups:
                df = pd.DataFrame({
                    "gene": deg_result["names"][group],
                    "logfoldchanges": deg_result["logfoldchanges"][group],
                    "pvals": deg_result["pvals"][group],
                    "pvals_adj": deg_result["pvals_adj"][group],
                    "scores": deg_result["scores"][group],
                })
                df.to_excel(writer, sheet_name=f"cluster_{group}", index=False)

        # UMAP - Expression level 
        sc.pl.umap(
            subset,
            color="IFNG",  
            cmap="Reds",  
            show=True,
            save=f"_{name}_subset_IFNG_expression_UMAP.pdf"
        )


        # dotplot - cluster
        dp = sc.pl.dotplot(
            subset, 
            var_names=existing_genes, 
            groupby="leiden", 
            standard_scale='var', 
            color_map="GnBu", 
            show=True, 
            return_fig=True
        )
        dp.savefig(f"{cwd}/figures/{name}_subset_cluster_{resolution}_dotplot.pdf")
        

        # ======= selected gene-positive cells vs rest =======
        positive_name_list = [
            "TRA+", "TRB+", "TRA+_TRB+", "(TRA+_or_TRB+)", "(TRA+_or_TRB+)_CD4-_CD8+", "(TRA+_or_TRB+)_CD4+_CD8-", "CD4+", "CD8+", "CD4+_CD8-", "CD4-_CD8+"
        ]

        for positive in positive_name_list:

            #UMAP
            min_cells_threshold = 3  
            true_count = subset.obs[f"{positive}"].sum()
            false_count = len(subset.obs[f"{positive}"]) - true_count

            if true_count < min_cells_threshold:
                print(f"Skipping {positive} due to insufficient positive cells (True: {true_count})")
                continue

            if false_count < min_cells_threshold:
                print(f"Skipping {positive} due to insufficient negative cells (False: {false_count})")
                continue

            else:
                print(f"True: {true_count}  False: {false_count}")
                sub = subset.copy()
                sub.obs[f"{positive}"] = pd.Categorical(sub.obs[f"{positive}"].astype(str),categories=["False", "True"])

                order = sub.obs[f"{positive}"] == "True"
                na_cells = sub.obs[~order].index
                true_cells = sub.obs[order].index

                new_order = na_cells.tolist() + true_cells.tolist()

                sub = sub[new_order, :]

                sc.pl.umap(
                    sub, 
                    color=f"{positive}",
                    palette=["lightgray", "blue"], 
                    show=True, 
                    save=f"_{name}_subset_{positive}_vs_rest_UMAP.pdf"
                )
                
                dp = sc.pl.dotplot(
                    sub, 
                    var_names=existing_genes, 
                    groupby=f"{positive}", 
                    standard_scale='var', 
                    color_map="GnBu", 
                    show=True, 
                    return_fig=True
                )
                dp.savefig(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_dotplot.pdf")

                dp = sc.pl.dotplot(
                    sub, 
                    var_names=CD4_genes, 
                    groupby=f"{positive}", 
                    standard_scale='var', 
                    color_map="GnBu", 
                    show=True, 
                    return_fig=True
                )
                dp.savefig(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_dotplot_CD4.pdf")

                dp = sc.pl.dotplot(
                    sub, 
                    var_names=CD8_genes, 
                    groupby=f"{positive}", 
                    standard_scale='var', 
                    color_map="GnBu", 
                    show=True, 
                    return_fig=True
                )
                dp.savefig(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_dotplot_CD8.pdf")

                custom_palette = {"True": "#1f77b4", "False": "#B0B0B0"}

                sc.pl.violin(
                    sub, 
                    keys=CD4_genes, 
                    groupby=f"{positive}", 
                    palette=custom_palette, 
                    stripplot=True, 
                    jitter=0.25, 
                    size=2, 
                    rotation=90, 
                    show=True, 
                    save=f"_{name}_subset_{positive}_vs_rest_violinplot_CD4.pdf"
                )

                sc.pl.violin(
                    sub, 
                    keys=CD8_genes, 
                    groupby=f"{positive}", 
                    palette=custom_palette, 
                    stripplot=True, 
                    jitter=0.25, 
                    size=2, 
                    rotation=90, 
                    show=True, 
                    save=f"_{name}_subset_{positive}_vs_rest_violinplot_CD8.pdf"
                )

                sc.tl.rank_genes_groups(
                    sub, 
                    groupby=f"{positive}", 
                    method="wilcoxon"
                )

                sc.pl.rank_genes_groups(
                    sub, 
                    n_genes=25, 
                    sharey=False, 
                    show=False, 
                    save=f"_{name}_subset_{positive}_vs_rest_DEG_fig.pdf"
                )

                deg_result = sub.uns["rank_genes_groups"]
                groups = deg_result["names"].dtype.names

                with pd.ExcelWriter(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_DEG_results.xlsx") as writer:
                    for group in groups:
                        df = pd.DataFrame({
                            "gene": deg_result["names"][group],
                            "logfoldchanges": deg_result["logfoldchanges"][group],
                            "pvals": deg_result["pvals"][group],
                            "pvals_adj": deg_result["pvals_adj"][group],
                            "scores": deg_result["scores"][group],
                        })
                        df.to_excel(writer, sheet_name=f"cluster_{group}", index=False)


                group = "True"
                df_deg = pd.DataFrame({
                    "gene": deg_result["names"][group],
                    "logfoldchanges": deg_result["logfoldchanges"][group],
                    "pvals": deg_result["pvals"][group],
                    "pvals_adj": deg_result["pvals_adj"][group],
                    "scores": deg_result["scores"][group],
                })

                df_deg["-log10_pval"] = -np.log10(df_deg["pvals"])
                df_deg["significant"] = (df_deg["pvals"] < 0.05) & (abs(df_deg["logfoldchanges"]) > 2)

                def plot_volcano(
                        df, 
                        name, 
                        cwd,
                        label_genes=True, 
                        xlim=None, 
                        ylim=None, 
                        label_specific_genes=None, 
                        label_suffix=None, 
                        base_ns_color="#B0B0B0", 
                        base_sig_color="#d62728", 
                        highlight_color="#1f77b4"
                    ):

                    df = df.copy()
                    if label_specific_genes is not None:
                        hi = {g.upper() for g in label_specific_genes}
                        df["is_highlight"] = df["gene"].str.upper().isin(hi)
                        genes_to_label = df[df["is_highlight"]]
                        if label_suffix is None:
                            label_suffix = "specificgenes"

                    elif label_genes:
                        df["is_highlight"] = df["significant"]
                        genes_to_label = df[df["significant"]]
                        if label_suffix is None:
                            label_suffix = "label"

                    else:
                        df["is_highlight"] = False
                        genes_to_label = pd.DataFrame()
                        if label_suffix is None:
                            label_suffix = "nolabel"

                    df["_ns"]  = ~df["significant"]
                    df["_sig"] = df["significant"] & ~df["is_highlight"]
                    df["_hi"]  = df["is_highlight"]

                    def _draw(ax, fix_range=False):
                        point_size = 24  

                        ax.scatter(
                            df.loc[df["_ns"], "logfoldchanges"],
                            df.loc[df["_ns"], "-log10_pval"],
                            c=base_ns_color, s=point_size, alpha=0.7, edgecolors="none", zorder=1
                        )
                        ax.scatter(
                            df.loc[df["_sig"], "logfoldchanges"],
                            df.loc[df["_sig"], "-log10_pval"],
                            c=base_sig_color, s=point_size, alpha=0.9, edgecolors="none", zorder=2
                        )
                        ax.scatter(
                            df.loc[df["_hi"], "logfoldchanges"],
                            df.loc[df["_hi"], "-log10_pval"],
                            c=highlight_color, s=point_size, alpha=1.0, edgecolors="none", zorder=4
                        )

                        ax.axhline(-np.log10(0.05), linestyle='--', color='black', linewidth=0.8)
                        ax.axvline(-2, linestyle='--', color='black', linewidth=0.8)
                        ax.axvline( 2, linestyle='--', color='black', linewidth=0.8)

                        if not genes_to_label.empty:
                            to_annot = genes_to_label.sort_values("-log10_pval", ascending=False)
                            offsets = [(10, 10), (10, -10), (-10, 10), (-10, -10)]  
                            for i, (_, r) in enumerate(to_annot.iterrows()):
                                dx, dy = offsets[i % len(offsets)]
                                x0, y0 = r["logfoldchanges"], r["-log10_pval"]
                                x1, y1 = x0 + dx * 0.05, y0 + dy * 0.05

                                ax.text(
                                    x1, y1, r["gene"],
                                    fontsize=9, weight="bold", color="black", zorder=6,
                                    path_effects=[pe.withStroke(linewidth=3, foreground="white")]
                                )
                                ax.plot([x0, x1], [y0, y1], color="black", linewidth=0.8, zorder=5)

                        ax.set_xlabel("log2 Fold Change")
                        ax.set_ylabel("-log10(p-value)")
                        ax.set_title(f"{name}_subset ({positive} vs rest)")

                        if fix_range:
                            if xlim is not None: ax.set_xlim(xlim)
                            if ylim is not None: ax.set_ylim(ylim)
                            ax.autoscale(enable=False)
                            ax.margins(x=0, y=0)

                    plt.figure(figsize=(8, 6))
                    ax = plt.gca()
                    _draw(ax, fix_range=False)
                    plt.tight_layout()
                    plt.savefig(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_volcano_plot_{label_suffix}.pdf")
                    plt.close()

                    fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=False)
                    _draw(ax, fix_range=True)
                    fig.savefig(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_volcano_plot_{label_suffix}_fixedrange.pdf", bbox_inches="tight")
                    plt.close(fig)



                plot_volcano(
                    df_deg, 
                    name, 
                    cwd, 
                    label_genes=True, 
                    xlim=[-10, 10]
                )

                plot_volcano(
                    df_deg, 
                    name, 
                    cwd, 
                    label_genes=False, 
                    xlim=[-10, 10]
                )

                specificgenesCD4 = [
                    "CD109", "TLR3", "TRAC", "CD4", "CCL2","CXCL16", "GBP4", "NOTCH1", "IFITM1", "SATB1","IRF7", "ISG15", "VCAM1", "JAG1", "SEMA4C","IFI6", "IFIT5", "IFI35", "MX2", "IFIH1"
                ]
                plot_volcano(
                    df_deg,
                    name, 
                    cwd, 
                    label_genes=False, 
                    xlim=[-10, 10], 
                    label_specific_genes=specificgenesCD4, 
                    label_suffix = "specificgenesCD4"
                )

                specificgenesCD8 = [
                    "CD8B", "TRAC", "CD8A", "SEMA4A", "CCL4", "CCL5","TOX", "IFI44", "JAML", "OAS1", "ISG15"
                ]
                plot_volcano(
                    df_deg, 
                    name, 
                    cwd, 
                    label_genes=False, 
                    xlim=[-10, 10], 
                    label_specific_genes=specificgenesCD8, 
                    label_suffix = "specificgenesCD8"
                )


        # ===================== TRA+_TRB+ cells vs. cells positive for the specified gene ====================

        print(subset.obs["TRA+"].dtype)
        print(subset.obs["TRA+"].unique())
        print(subset.obs["TRB+"].dtype)
        print(subset.obs["TRB+"].unique())

        subset.obs["TRA+_TRB+"] = (subset.obs["TRA+"]) & (subset.obs["TRB+"])
        subset.obs["TRA+_TRB-"] = (subset.obs["TRA+"]) & ~(subset.obs["TRB+"])
        subset.obs["TRA-_TRB+"] = ~(subset.obs["TRA+"]) & (subset.obs["TRB+"])
        subset.obs["TRA-_TRB-"] = ~(subset.obs["TRA+"]) & ~(subset.obs["TRB+"])

        print(f"TRA+_TRB+ counts:\n{subset.obs['TRA+_TRB+'].value_counts()}")
        print(f"TRA+_TRB- counts:\n{subset.obs['TRA+_TRB-'].value_counts()}")
        print(f"TRA-_TRB+ counts:\n{subset.obs['TRA-_TRB+'].value_counts()}")
        print(f"TRA-_TRB- counts:\n{subset.obs['TRA-_TRB-'].value_counts()}")


        subset.obs["group"] = "other"  
        subset.obs.loc[subset.obs["TRA+_TRB+"], "group"] = "TRA+_TRB+"
        subset.obs.loc[subset.obs["TRA+_TRB-"], "group"] = "TRA+_TRB-"
        subset.obs.loc[subset.obs["TRA-_TRB+"], "group"] = "TRA-_TRB+"
        subset.obs.loc[subset.obs["TRA-_TRB-"], "group"] = "TRA-_TRB-"

        groupname_list = ["TRA+_TRB-", "TRA-_TRB+", "TRA-_TRB-"]

        for groupname in groupname_list:

            min_cells_threshold = 3
            group_counts = subset.obs["group"].value_counts()
            if group_counts.get("TRA+_TRB+", 0) < min_cells_threshold:
                print(f"Skipping due to insufficient positive cells (TRA+_TRB+: {group_counts.get('TRA+_TRB+', 0)})")
                continue  

            if group_counts.get(groupname, 0) < min_cells_threshold:
                print(f"Skipping due to insufficient negative cells ({groupname}: {group_counts.get(groupname, 0)})")
                continue  
            else:

                subset = subset[subset.obs["group"].sort_values().index, :]
                sc.pl.umap(
                    subset, 
                    color="group", 
                    palette=["red", "blue", "green", "lightgray"], 
                    show=True, 
                    save=f"_{name}_subset_TRA+_TRB+_vs_{groupname}_UMAP.pdf"
                )

                subset_filtered = subset[subset.obs["group"].isin(["TRA+_TRB+", groupname]), :]


                sc.tl.rank_genes_groups(
                    subset_filtered, 
                    groupby="group", 
                    groups=["TRA+_TRB+", groupname], 
                    method="wilcoxon"
                )
                sc.pl.rank_genes_groups(
                    subset_filtered, 
                    n_genes=25, 
                    sharey=False, 
                    show=False, 
                    save=f"_{name}_subset_TRA+_TRB+_vs_{groupname}_DEG_fig.pdf"
                )

                deg_result = subset_filtered.uns["rank_genes_groups"]
                groups = deg_result["names"].dtype.names
                with pd.ExcelWriter(f"{cwd}/figures/{name}_subset_TRA+_TRB+_vs_{groupname}_DEG_results.xlsx") as writer:
                    for group in groups:
                        df = pd.DataFrame({
                            "gene": deg_result["names"][group],
                            "logfoldchanges": deg_result["logfoldchanges"][group],
                            "pvals": deg_result["pvals"][group],
                            "pvals_adj": deg_result["pvals_adj"][group],
                            "scores": deg_result["scores"][group],
                        })
                        df.to_excel(writer, sheet_name=f"{group[:31]}", index=False)


                # --- dotplot ---
                dp = sc.pl.dotplot(
                    subset_filtered, 
                    var_names=existing_genes, 
                    groupby="group", 
                    standard_scale='var', 
                    color_map="GnBu", 
                    show=True, 
                    return_fig=True
                )
                dp.savefig(f"{cwd}/figures/{name}_subset_TRA+_TRB+_vs_{groupname}_dotplot.pdf")


                # --- volcano plot ---
                deg_result = subset_filtered.uns["rank_genes_groups"]
                group = "TRA+_TRB+"
                df_deg = pd.DataFrame({
                    "gene": deg_result["names"][group],
                    "logfoldchanges": deg_result["logfoldchanges"][group],
                    "pvals": deg_result["pvals"][group],
                    "pvals_adj": deg_result["pvals_adj"][group],
                    "scores": deg_result["scores"][group],
                })
                
                exclude_genes = ["TRAC", "TRBC1", "TRBC2"]
                df_deg = df_deg[~df_deg["gene"].isin(exclude_genes)].copy()

                df_deg["-log10_pval"] = -np.log10(df_deg["pvals"])
                df_deg["significant"] = (df_deg["pvals"] < 0.05) & (abs(df_deg["logfoldchanges"]) > 2)

                def plot_volcano(
                        df, 
                        name, 
                        cwd, 
                        label_genes=True, 
                        xlim=[-10, 10], 
                        ylim=None
                    ):

                    plt.figure(figsize=(8, 6))
                    plt.scatter(
                        df["logfoldchanges"], df["-log10_pval"],
                        c=df["significant"].map({True: "red", False: "gray"}),
                        alpha=0.7, edgecolors="none"
                    )
                    if label_genes:
                        for _, row in df[df["significant"]].iterrows():
                            plt.text(
                                row["logfoldchanges"],
                                row["-log10_pval"],
                                row["gene"],
                                fontsize=8,
                                ha='center',
                                va='bottom'
                            )
                    plt.axhline(-np.log10(0.05), linestyle='--', color='black', linewidth=0.8)  
                    plt.axvline(-2, linestyle='--', color='black', linewidth=0.8)  
                    plt.axvline(2, linestyle='--', color='black', linewidth=0.8)   
                    plt.xlabel("log2 Fold Change")
                    plt.ylabel("-log10(p-value)")
                    plt.title(f"{name}_subset ({positive} vs rest)")
                    plt.tight_layout()

                    label_suffix = "label" if label_genes else "nolabel"
                    plt.savefig(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_volcano_plot_{label_suffix}.pdf")
                    plt.close()

                    fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=False)

                    plt.scatter(
                        df["logfoldchanges"], df["-log10_pval"],
                        c=df["significant"].map({True: "red", False: "gray"}),
                        alpha=0.7, edgecolors="none"
                    )
                    if label_genes:
                        for _, row in df[df["significant"]].iterrows():
                            plt.text(
                                row["logfoldchanges"],
                                row["-log10_pval"],
                                row["gene"],
                                fontsize=8,
                                ha='center',
                                va='bottom',
                                clip_on=True
                            )

                    plt.axhline(-np.log10(0.05), linestyle='--', color='black', linewidth=0.8)
                    plt.axvline(-2, linestyle='--', color='black', linewidth=0.8)
                    plt.axvline(2, linestyle='--', color='black', linewidth=0.8)
                    plt.xlabel("log2 Fold Change")
                    plt.ylabel("-log10(p-value)")
                    plt.title(f"{name}_subset ({positive} vs rest)")

                    if xlim is not None:
                        plt.xlim(xlim)
                    if ylim is not None:
                        plt.ylim(ylim)
                    ax.autoscale(enable=False)  
                    ax.margins(x=0, y=0) 

                    plt.savefig(f"{cwd}/figures/{name}_subset_{positive}_vs_rest_volcano_plot_{label_suffix}_fixedrange.pdf", bbox_inches="tight")
                    plt.close()

                plot_volcano(df_deg, name, cwd, label_genes=True)
                plot_volcano(df_deg, name, cwd, label_genes=False)

    except Exception as e:
        print(f"Error in {name}: {e}")


In [ ]:
# <<<<<<< gene expression dotplot (subset) >>>>>>>

existing_genes = [g for g in human_gene_list if g in adata.var_names]

missing_genes = [g for g in human_gene_list if g not in adata.var_names]

if missing_genes:
    print("The following genes were not found (excluded):")
    print(missing_genes)

subsets_of_a_cell = []

for name, subset in subsets:
    subset_cp = subset.copy()
    subset_cp.obs["subset_name"] = name
    subset_cp.obs_names = [f"{name}_{i}" for i in subset_cp.obs_names]  
    subsets_of_a_cell.append(subset_cp)

adata_subsets_of_a_cell = ad.concat(subsets_of_a_cell, join="outer")

dp = sc.pl.dotplot(
    adata_subsets_of_a_cell,
    var_names=existing_genes,
    groupby="subset_name",
    standard_scale='var',
    color_map="GnBu",
    show=True,
    return_fig=True
)

dp.savefig(f"{cwd}/figures/dotplot_by_subset.pdf")


# Rename in Tarminal

for f in *mouse*; do
  new=$(echo "$f" | sed 's/^.*mouse/mouse/')
  mv "$f" "$new"
done

# Rename in PowerShell

Get-ChildItem -File *mouse* | ForEach-Object {
    $newName = $_.Name -replace '.*mouse','mouse'
    if ($_.Name -ne $newName) {
        Rename-Item -LiteralPath $_.FullName -NewName $newName
        Write-Output "$($_.Name) → $newName"
    }
}